In [ ]:
from __future__ import print_function, division   # Python 2/3 compatibility
from skimage import io, img_as_ubyte              # utilities to read and write images in various formats
import numpy as np                                # array manipulation package
import matplotlib.pylab as plt                    # plotting package
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 16)         # set default figure size
plt.rcParams['image.cmap'] = 'gray'               # set default colormap to gray

In [ ]:
pip install opencv-python

# Assignment 2 : Intensity Transformations and Spatial Filtering

The following programming assignment involves two tasks, viz.: basic histogram processing and spatial domain image filtering tasks, i.e., image sharpening.

**Please, follow carefully the submission instructions given in the end of this notebook.** You are encouraged to seek information in other places than the course book and lecture material but remember **list all your sources under references**.

If you experience problems that you cannot solve using the course material or the Python documentation, or have any questions regarding the programming assignments, please do not hesitate to contact the course assistant by sending an e-mail at dip@unioulu.oulu.fi. You can also join in for the Q & A session (schedule is given on the course page in Moodle) for this assignment.

**At first, fill in your personal details below.**

# Personal details:

* **Name(s) and student ID(s):**
* Haseeb Ur Rehman (2315255) and Marwa Bibi (2407704)
* **Contact information:**
* Haseeb.Rehman@student.oulu.fi and Marwa.Bibi@student.oulu.fi

# 1. Histogram operations

In the following, you will have to analyze two images, `coffee.jpg` and `pout.tif`, and their histograms, and to compare the results of two histogram operations, namely histogram equalization and stretching. Now, perform the following operations in the reserved code cells and answer to the questions written in **bold** into the reserved spaces in **Finnish or English**.

**1.1. Read and display the images `coffee.jpg` and `pout.tif` and their histograms in the same figure.**

Hint: You can plot the histogram of an image with matplotlib's __[`hist()`](https://matplotlib.org/devdocs/api/_as_gen/matplotlib.pyplot.hist.html)__ function but please note that you have to ravel the pixels of the 2D image into 1D array first.

In [ ]:
# read the two images
coffee = (io.imread('coffee.jpg', as_gray=True) )  
coffee = img_as_ubyte(coffee) 
pout = io.imread('pout.tif')

# display the two images and their histograms in the same figure
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Coffee image and histogram
axes[0, 0].imshow(coffee, cmap='gray')
axes[0, 0].set_title('Coffee Image')
axes[0, 0].axis('off')
axes[0, 1].hist(coffee.ravel(), bins=256, range=(0, 255), color='black')
axes[0, 1].set_title('Coffee Histogram')


# Pout image and histogram
axes[1, 0].imshow(pout, cmap='gray')
axes[1, 0].set_title('Pout Image')
axes[1, 0].axis('off')
axes[1, 1].hist(pout.ravel(), bins=256, range=(0, 255), color='black')
axes[1, 1].set_title('Pout Histogram')

plt.tight_layout()
plt.show()


**Compare the two images and their histograms. What can you say about the contrast of the images?**

`According to Wikipedia, the contrast is the differnce in the color that makes the object to be different from other objects within the same field of view. In the coffee image, the contrast is low because values are concentarated in a anrrow range which results in minimum details. While, in the pout image, the contrast is higher because the pixel intensity values are distributed in a wider range so there is visible difference in the bright and dark areas.   `

### Histogram equalization

**1.2. Perform histogram equalization with the function __[`exposure.equalize_hist()`](http://scikit-image.org/docs/dev/api/skimage.exposure.html#skimage.exposure.equalize_hist)__ and display the resulting images and their histograms in the same figure.**

Hint: Please note that `exposure.equalize_hist()` function returns `float64` image. You need to __[convert the image back to `uint8`](http://scikit-image.org/docs/dev/user_guide/data_types.html)__ after histogram equalization so that the intensity value range of the resulting and original histograms are comparable.

In [ ]:
from skimage import exposure
from skimage import img_as_ubyte

# perform histogram equalization and convert data type from 'float64' back to 'uint8' after histogram equalization
# Perform histogram equalization
coffee_equalized = exposure.equalize_hist(coffee)
pout_equalized = exposure.equalize_hist(pout)
# Convert the equalized images back to uint8
coffee_equalized = img_as_ubyte(coffee_equalized)
pout_equalized = img_as_ubyte(pout_equalized)
# display resulting images and their histograms in the same figure
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Coffee equalized image and histogram
axes[0, 0].imshow(coffee_equalized, cmap='gray')
axes[0, 0].set_title('Coffee Equalized Image')
axes[0, 0].axis('off')
axes[0, 1].hist(coffee_equalized.ravel(), bins=256, range=(0, 255), color='black')
axes[0, 1].set_title('Coffee Equalized Histogram')

# Pout equalized image and histogram
axes[1, 0].imshow(pout_equalized, cmap='gray')
axes[1, 0].set_title('Pout Equalized Image')
axes[1, 0].axis('off')
axes[1, 1].hist(pout_equalized.ravel(), bins=256, range=(0, 255), color='black')
axes[1, 1].set_title('Pout Equalized Histogram')

plt.tight_layout()
plt.show()


**Again, compare the two images. Did histogram equalization help in increasing image contrast? Why or why not?**

`Histogram equalization represents a strong technique for enhancing image contrast which spreads pixel intensities across the entire [0, 255] dynamic range. Application of this technique produced outstanding results when implemented to the coffee image because its narrow ranged caused poor contrast before equalization. The image grew brighter together with increased detail content because the histogram expanded its coverage to span the whole available intensity spectrum. The pout image maintained its existing high contrast values before equalization while its intensity spread inadequately filled the complete range. Equalization provided additional contrast improvement to the darker parts yet its results were minute than the coffee image. But histogram equalization increased the contrast in both images`

### Contrast stretching

Another way of improving the contrast in an image is to simply stretch the original pixel values over an extended dynamic range using a linear scaling function. For instance, in case of an `uint8` image, the desired value range of a contrast-stretched image could be the full range from 0 to 255. 

Intuitively, one could perform contrast-stretching by selecting the minimum and maximum values of the original image and map these values to 0 and 255, respectively, and linearly scale all other pixel values in between accordingly. However, even a single outlier pixel value (high or low) can affect the input scaling range too much when outcome of the histogram stretching is not particularly good.

A more robust approach is to map the intensity values so that e.g. 1st and 99th percentiles of the histogram are saturated at the minimum and maximum values of the desired intensity range. In other words, 1% of the pixels of both low and high intensities will be mapped to 0 and 255 in the contrast-stretched image while rest are scaled linearly in between.

**1.3. Now, perform contrast stretching on the original images with the help of functions __[`np.percentile()`](https://numpy.org/doc/stable/reference/generated/numpy.percentile.html)__ and __[`exposure.rescale_intensity()`](http://scikit-image.org/docs/dev/api/skimage.exposure.html#skimage.exposure.rescale_intensity)__ so that the full range from 0 and 255 is utilized based on the 1st and 99th percentiles of their histograms. Then, display the resulting images and their histograms in the same figure.**

In [ ]:

# find the 1st and 99th percentiles of each image
coffee_p1, coffee_p99 = np.percentile(coffee, (1, 99))
pout_p1, pout_p99 = np.percentile(pout, (1, 99))

# rescale the intensities of both images to full 'uint8' range [0, 255] based on their 1st and 99th percentiles
coffee_stretched = exposure.rescale_intensity(coffee, in_range=(coffee_p1, coffee_p99))
pout_stretched = exposure.rescale_intensity(pout, in_range=(pout_p1, pout_p99))
# Convert the stretched images back to uint8
coffee_stretched = img_as_ubyte(coffee_stretched)
pout_stretched = img_as_ubyte(pout_stretched)

# display resulting images and their histograms
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Coffee stretched image and histogram
axes[0, 0].imshow(coffee_stretched, cmap='gray')
axes[0, 0].set_title('Coffee Stretched Image')
axes[0, 0].axis('off')
axes[0, 1].hist(coffee_stretched.ravel(), bins=256, range=(0, 255), color='black')
axes[0, 1].set_title('Coffee Stretched Histogram')

# Pout stretched image and histogram
axes[1, 0].imshow(pout_stretched, cmap='gray')
axes[1, 0].set_title('Pout Stretched Image')
axes[1, 0].axis('off')
axes[1, 1].hist(pout_stretched.ravel(), bins=256, range=(0, 255), color='black')
axes[1, 1].set_title('Pout Stretched Histogram')

plt.tight_layout()
plt.show()


### Comparison

**1.4. Finally, display the original `coffee.jpg` image, and its histogram-equalized and contrast-stretched versions and the corresponding histograms into one figure (in total six images in one figure). Do the same for `pout.tif` as well.**

In [ ]:
# 6 subplots for 'coffee.jpg'
# Display coffee.jpg and its processed versions
fig, axes = plt.subplots(3, 2, figsize=(10, 12))

# Original Coffee Image and Histogram
axes[0, 0].imshow(coffee, cmap='gray')
axes[0, 0].set_title('Original Coffee Image')
axes[0, 0].axis('off')
axes[0, 1].hist(coffee.ravel(), bins=256, range=(0, 255), color='black')
axes[0, 1].set_title('Original Coffee Histogram')

# Histogram Equalized Coffee Image and Histogram
axes[1, 0].imshow(coffee_equalized, cmap='gray') 
axes[1, 0].set_title('Equalized Coffee Image')
axes[1, 0].axis('off')
axes[1, 1].hist(coffee_equalized.ravel(), bins=256, range=(0, 255), color='black')
axes[1, 1].set_title('Equalized Coffee Histogram')

# Contrast Stretched Coffee Image and Histogram
axes[2, 0].imshow(coffee_stretched, cmap='gray')
axes[2, 0].set_title('Stretched Coffee Image')
axes[2, 0].axis('off')
axes[2, 1].hist(coffee_stretched.ravel(), bins=256, range=(0, 255), color='black')
axes[2, 1].set_title('Stretched Coffee Histogram')

plt.tight_layout()
plt.show()


# 6 subplots for 'pout.tif'
# Display pout.tif and its processed versions
fig, axes = plt.subplots(3, 2, figsize=(10, 12))

# Original Pout Image and Histogram
axes[0, 0].imshow(pout, cmap='gray')
axes[0, 0].set_title('Original Pout Image')
axes[0, 0].axis('off')
axes[0, 1].hist(pout.ravel(), bins=256, range=(0, 255), color='black')
axes[0, 1].set_title('Original Pout Histogram')

# Histogram Equalized Pout Image and Histogram
axes[1, 0].imshow(pout_equalized, cmap='gray')
axes[1, 0].set_title('Equalized Pout Image')
axes[1, 0].axis('off')
axes[1, 1].hist(pout_equalized.ravel(), bins=256, range=(0, 255), color='black')
axes[1, 1].set_title('Equalized Pout Histogram')

# Contrast Stretched Pout Image and Histogram
axes[2, 0].imshow(pout_stretched, cmap='gray')
axes[2, 0].set_title('Stretched Pout Image')
axes[2, 0].axis('off')
axes[2, 1].hist(pout_stretched.ravel(), bins=256, range=(0, 255), color='black')
axes[2, 1].set_title('Stretched Pout Histogram')

plt.tight_layout()
plt.show()

**Which method gives better result for each of the two images in** ***your*** **opinion? Why??**

`The coffee image gave better reulsts in histogram equalization since it enhances contrast levels through uniform pixel distribution which reveals hidden details in low-contrast images. While, for the pout image, contrast stretching is good since it optimizes contrast without distorting either natural appearance or original histogram shapes unlike the over-enhancement of histogram equalization. The selected method for image enhancement depends on the initial contrast because histogram equalization works best on images with low contrast and contrast stretching works optimally on images with moderate to high initial contrast strength.`

# 2. Image sharpening

First, read the part concerning sharpening spatial transforms in the lecture notes or in the course book.

In this exercise, your task is to perform a sharpening transform to the image `moonunsharp.tif` in spatial domain enhancing the details, like edges, in the original grayscale image. The use of built-in functions that perform image sharpening from scratch, like `scipy.misc.imfilter()`, is forbidden but functions like __[`scipy.signal.convolve2d()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve2d.html)__ can be used for the task. You can select some method presented in the lecture notes or the course book, e.g. use Laplacian operator and convolution, for sharpening the test image. 

Please note that it does not matter what method you use or how “good” the sharpening looks as long as the sharpening can be observed in the end result. An example result achieved with __[`ImageFilter`](https://pillow.readthedocs.io/en/stable/reference/ImageFilter.html)__ is shown below:

In [ ]:
# image sharpening example using 'ImageFilter' module from PILLOW with 'image.filter()' function
from PIL import ImageFilter, Image

moon = Image.open('moonunsharp.tif')
moon_sharp_example = moon.filter(ImageFilter.SHARPEN)

fig, ax = plt.subplots(1, 2)
ax[0].imshow(moon, vmin=0, vmax=255, cmap=plt.get_cmap('gray'))
ax[0].set_title('original')
ax[0].axis('off')
ax[1].imshow(moon_sharp_example, vmin=0, vmax=255, cmap=plt.get_cmap('gray'))
ax[1].set_title('sharpened')
ax[1].axis('off')
fig.tight_layout()

**2.1. Now, implement your own image sharpening transform and apply it on the test image.**

Hint: Like in the previous task, please note the __[image data type (`dtype`) and corresponding value range](http://scikit-image.org/docs/dev/user_guide/data_types.html)__ after filtering/sharpening as unexpected errors with arithmetic may occur (see pre-tutorials) !

In [ ]:
# Perform image sharpening using e.g. a Laplacian mask and convolution
# Perform image sharpening using e.g. a Laplacian mask and convolution
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.signal import convolve2d

# Load the image in grayscale
image = cv2.imread('moonunsharp.tif', cv2.IMREAD_GRAYSCALE)

# Define the Laplacian kernel
laplacian_kernel = np.array([[0,  1,  0],
                             [1, -4,  1],
                             [0,  1,  0]])

# Perform convolution using the Laplacian kernel
laplacian_filtered = convolve2d(image, laplacian_kernel, mode='same', boundary='symm')

# Apply Laplacian mask to enhance edges
laplacian_mask = cv2.filter2D(image, ddepth=-1, kernel=laplacian_kernel)

# Sharpen the image by subtracting the Laplacian result from the original image
sharpened_image = image - laplacian_filtered

# Clip values to be in valid range (0-255)
sharpened_image = np.clip(sharpened_image, 0, 255).astype(np.uint8)

**2.2. Display the original and sharpened moon images in the same figure.**

In [ ]:
# Display the results
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.title("Original Image")
plt.imshow(image, cmap='gray')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title("Laplacian Mask")
plt.imshow(laplacian_mask, cmap='gray')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.title("Sharpened Image")
plt.imshow(sharpened_image, cmap='gray')
plt.axis('off')

plt.show()

# Aftermath
Finally, fill your answers to the following questions:

**How much time did you need to complete this exercise?**

`Around continuous 5-6 hours in understanding and implementation `

**Did you experience any problems with the exercise? Was there enough help available? Should this notebook be more (or less) detailed?**

`The exercise was helpful along with som external useful links and off course with the lecture topics`

# References

`https://www.geeksforgeeks.org/opencv-python-program-analyze-image-using-histogram/ https://theailearner.com/2019/01/30/what-is-contrast-in-image-processing/ https://scikit-image.org/docs/dev/api/skimage.exposure.html#skimage.exposure.equalize_hist https://www.geeksforgeeks.org/histogram-equalization-in-digital-image-processing/ https://samirkhanal35.medium.com/contrast-stretching-f25e7c4e8e33 https://www.youtube.com/watch?v=5l0y-LMM1c0`

# Submission

1. Before submitting your work, **check that your notebook (code) runs from scratch** and reproduces all the requested results by clicking on the menu `Kernel -> Restart & Run All`! Also, check that you have answered all the questions written in **bold**.
2. Clear all outputs and variables, etc. by click on the menu `Kernel -> Restart & Clear Output`. This may (or will) reduce the file size of your deliverable a lot! 
3. Rename this Jupyter notebook to **`DIP_PA2_[student number(s)].ipynb`** (e.g. `DIP_PA2_1234567.ipynb` if solo work or `DIP_PA2_1234567-7654321.ipynb` if working in a pair)